# Assignment 1-2: Data Collection Using Web APIs

## Objective

Many Websites (such as Twitter, Yelp, Spotify) provide free APIs to allow users to access their data. *API wrappers* simplify the use of these APIs by wrapping API calls into easy-to-use Python functions. At SFU, we are developing a unified API wrapper, called [DataPrep.Connector](https://docs.dataprep.ai/user_guide/connector/introduction.html#userguide-connector), which offers a unified programming interface to collect data from a variety of Web APIs.

In this assignment, you will learn the following:

* How to ask insightful questions about data
* How to collect data from Web APIs using DataPrep.Connector

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

DataPrep.Connector is very easy to learn. By watching this 10-min [PyData Global 2020 talk](https://www.youtube.com/watch?v=56qu-0Ka-dA), you should be able to know how to use it. 

If you want to know more, below are some other useful resources.

* [Quick Introduction](https://github.com/sfu-db/dataprep#connector)
* [User Guide](https://sfu-db.github.io/dataprep/user_guide/connector/connector.html) 
* [Examples](https://github.com/sfu-db/dataprep/tree/develop/examples)
* [Fetch and analyze COVID-19 tweets using DataPrep](https://www.youtube.com/watch?v=vvypQB3Vp1o)

## Overview

This is a **group** assignment. 
Please check your group in this [PDF file](https://coursys.sfu.ca/2022sp-cmpt-733-g1/pages/Web-API-Assignment/view).

To do this assignment, your group needs to go through four steps:

1. Select a new Web API that is not listed on https://github.com/sfu-db/APIConnectors. 
2. Create a configuration file for the API (see tutorials at [link1](https://github.com/sfu-db/APIConnectors/blob/develop/CONTRIBUTING.md#add-configuration-files) and [link2](https://github.com/sfu-db/EZHacks-tutorial/blob/master/2.%20Tutorial.ipynb)). 
3. Come up with four questions about the API. 
4. Write code to answer these questions one by one.

For Step 3, please make sure your questions are **good**.

## What are "good questions"?

Please use the following to judge whether your questions are good or not.

1. Good questions need to be useful. That is, they are either common questions asked about the API or are exploring novel use cases.
2. Good questions need to be diverse. That is, they cover different aspects of the API. 
3. Good questions have to cover different difficulty levels. That is, it consists of both easy and difficult questions,  where the difficulty can be measured by the number of lines of code or the number of input parameters.

The following shows four good questions about the Yelp API. The corresponding code can be found at this [link](https://github.com/sfu-db/DataConnectorConfigs#yelp----collect-local-business-data).

* Q1. What's the phone number of Capilano Suspension Bridge Park?
* Q2. Which yoga store has the highest review count in Vancouver?
* Q3. How many Starbucks stores are in Seattle and where are they?
* Q4. What are the ratings for a list of restaurants?

**Why are they useful?**
* Q1 is useful because "Capilano Suspension Bridge Park" is one of the most popular tourist attractions in Vancouver.
* Q2 is useful because a yoga fan wants to find out the most popular yoga store in Vancouver. 
* Q3 is useful because Starbucks was founded in Seattle.
* Q4 is useful because people often rely on yelp ratings to decide which restaurant to go to.

**Why are they diverse?**

This is because the [code](yelp-code.png) written to answer them has different inputs or outputs.
* Q1 takes `term` and `location` as input and returns 1 record with attributes `name` and `phone` 
* Q2 takes `categories`, `location`, and `sort_by` as input and returns 1 record with attributes `name` and `review_count`
* Q3 takes `term` and `location` as input and returns n records with attributes `name`, `address`, `city`, `state`, `country`, `zipcode`
* Q4 takes a list of retarurant `names` as input and return n records with attributes `name`, `rating`, `city`

**Why are they more and more difficult?**
* Q1 (4 lines of code, 2 query parameters)
* Q2 (4 lines of code, 3 query parameters)
* Q3 (5 lines of code, 2 query parameters)
* Q4 (11 lines of code, 2 query parameters)

Please note that you have to use DataPrep.Connector to get data from the Web API. If DataPrep.Connector cannot meet your needs, please post your questions on Teams (Channel: Assignment 1). We will help you. 

## Now, it's your turn. :) 

Please write down your questions and the corresponding code for your assigned API. 

In [1]:
## Provide your API key here for TAs to reproduce your results
import pandas as pd
from dataprep.connector import connect

APIKEY = 'af093fe5ad9cc1c2125e2464c51b4edd'

conn_tmdb = connect("./tmdb", _auth={"access_token":APIKEY})

# Common function to get genres from a list of genre IDs

genres = await conn_tmdb.query('movie_genres')
genres_dict = {}
for index, row in genres.iterrows():
    genres_dict[row['id']] = row['name']
    
def get_genres_list(ids):
    names = []
    for id in ids:
        names.append(genres_dict[id])
    return names

### Q1 What is the release date of the movie "Top Gun: Maverick"

In [2]:
query = "Top Gun: Maverick"
movie = await conn_tmdb.query('search_movies', query=query)
movie

,original_title,release_date
0,Top Gun: Maverick,2022-05-27


### Q2 What are the top 10 popular movies playing now?

In [3]:
# parameter region accept a ISO 3166-1 value.  https://en.wikipedia.org/wiki/ISO_3166-1
now_playing = await conn_tmdb.query('now_playing', region="CA")
now_playing['genres']=now_playing['genre_ids'].map(lambda x:get_genres_list(x))
df = now_playing.drop(columns=['genre_ids'])\
                .sort_values(by="popularity", ascending=False)\
                .head(10)
df

,title,release_date,popularity,vote_average,vote_count,genres
0,Puss in Boots: The Last Wish,2022-12-21,9724,8.6,1322,"[Animation, Action, Adventure, Comedy, Family,..."
1,Violent Night,2022-12-02,2254,7.7,889,"[Action, Comedy, Crime, Thriller]"
2,M3GAN,2023-01-06,2067,7.2,201,"[Science Fiction, Horror, Comedy]"
4,The Pale Blue Eye,2022-12-23,637,7.1,495,"[Mystery, Thriller, Horror, Crime]"
3,Top Gun: Maverick,2022-05-27,519,8.3,5589,"[Action, Drama]"
5,A Man Called Otto,2023-01-06,294,7.0,43,"[Comedy, Drama]"
6,The Quintessential Quintuplets Movie,2022-12-02,176,8.7,151,"[Animation, Comedy, Romance]"
7,Babylon,2022-12-23,132,7.5,47,"[Drama, Comedy]"
8,The Whale,2022-12-16,108,7.4,23,[Drama]
10,Women Talking,2022-12-23,39,8.1,9,[Drama]


### Q3 What are the movies of "Viggo Mortensen" and "Liv Tyler" act together?

In [4]:
actor1_df = await conn_tmdb.query("search_person", query="Viggo Mortensen")
actor2_df = await conn_tmdb.query("search_person", query="Liv Tyler")
movie_credit_1_df = await conn_tmdb.query("movie_credit", person_id=actor1_df['id'][0])
movie_credit_1_df = movie_credit_1_df.drop(columns=['genre_ids'])
movie_credit_2_df = await conn_tmdb.query("movie_credit", person_id=actor2_df['id'][0])
movie_credit_2_df = movie_credit_2_df.drop(columns=['genre_ids'])
result = pd.merge(movie_credit_1_df, movie_credit_2_df, how="inner")
result

,movie_id,title,vote_average,overview,popularity,release_date
0,120,The Lord of the Rings: The Fellowship of the Ring,8.391,"Young hobbit Frodo Baggins, after inheriting a...",95.532,2001-12-18
1,121,The Lord of the Rings: The Two Towers,8.374,Frodo and Sam are trekking to Mordor to destro...,78.134,2002-12-18
2,122,The Lord of the Rings: The Return of the King,8.470,Aragorn is revealed as the heir to the ancient...,87.679,2003-12-01
3,71506,Ringers: Lord of the Fans,6.400,'Ringers: Lord of the Fans' is a feature-lengt...,3.657,2005-01-21
4,445652,Quest for the Ring,6.200,,1.141,2001-11-29
5,453779,A Passage to Middle-earth: Making of 'Lord of ...,6.400,,2.786,2001-12-09
6,37757,Never Sleep Again: The Elm Street Legacy,7.929,"For decades, Freddy Krueger has slashed his wa...",12.119,2010-05-04


### Q4 Tom Cruise's Top 5 Action Movies?

In [5]:
actor = await conn_tmdb.query("search_person", query="Tom Cruise")
actor_id = actor['id'][0]
movies = await conn_tmdb.query("movie_credit", person_id=actor_id)
movies['genres'] = movies['genre_ids'].map(lambda x: ', '.join(get_genres_list(x)))
movies = movies[movies['genres'].str.contains('Action')]
top_rated = movies.sort_values(by="vote_average", ascending=False).head(5)
top_rated = top_rated[['title', 'genres', 'vote_average', 'release_date']]
top_rated

,title,genres,vote_average,release_date
42,Top Gun: Maverick,"Action, Drama",8.334,2022-05-27
31,Edge of Tomorrow,"Action, Science Fiction",7.597,2014-05-27
3,The Last Samurai,"Drama, Action, War",7.561,2003-12-05
34,Mission: Impossible - Fallout,"Action, Adventure",7.400,2018-07-13
1,Minority Report,"Action, Thriller, Science Fiction, Mystery",7.334,2002-06-20


## Submission

Complete this notebook, rename it to `A1-2-[WEB API Name].ipynb`, and submit it along with your config files to the CourSys activity `Assignment 1 - Part 2`. For example, if your group works on Yelp, then **every member of your group** needs to submit the same notebook named `A1-2-Yelp.ipynb` and the config files named `config.zip`.